#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Todos

* ~~create an agent~~
* ~~connect it to Ollama~~
* connect it to a data store
* be able to CRUD
* ~~be able converse in turns~~ use `input`


In [14]:
from pydantic import BaseModel, Field

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]

class Story(BaseModel):
    id: str
    genre: str
    logline: str
    summary: str
    plotPoints: list[str]
    characters: list[Character]
    acts: list[Beat]


In [ ]:
from typing import Generic, TypeVar

T = TypeVar("T", bound=BaseModel)

class JsonModelStore(Generic[T]):
    def __init__(self, path: Path | str, model_cls: Type[T]):
        self.path = Path(path)
        self.model_cls = model_cls
        self.path.parent.mkdir(parents=True, exist_ok=True)
        if not self.path.exists():
            # Create a default (empty) file
            self._write(self.model_cls.model_validate({}).model_dump())

    def _read(self) -> dict[str, Any]:
        try:
            return json.loads(self.path.read_text(encoding="utf-8"))
        except (FileNotFoundError, json.JSONDecodeError):
            return {}

    def _write(self, data: dict[str, Any]) -> None:
        self.path.write_text(json.dumps(data, indent=2), encoding="utf-8")

    def get(self) -> T:
        return self.model_cls.model_validate(self._read())

    def update(self, **updates: Any) -> None:
        merged = {**self._read(), **updates}
        self._write(merged)

    def delete(self) -> None:
        self.path.unlink(missing_ok=True)

    def exists(self) -> bool:
        return self.path.exists()


In [3]:
from langchain_ollama.chat_models import ChatOllama

my_agent = ChatOllama(
    model="gpt-oss:20b"
)

In [ ]:
from langchain.tools import tool

@tool
def write_to_database():
    pass

@tool
def read_from_database():
    pass



In [12]:
from langchain.messages import SystemMessage, HumanMessage
from IPython.display import Markdown, display

messages = [
        SystemMessage("You are an expert in screenwriting. Make your answers only so long as it is necessary. You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative."),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    messages.append(HumanMessage(user_input))
    
    response = my_agent.invoke(messages)
    
    messages.append(response)
    
    display(Markdown(response.content))

**Logline**  
Two down‑to‑earth, rural German lads crash into Berlin’s techno‑twisted, art‑laden dating world.  Every attempt at romance turns into a slap‑stick mis‑translation, a missed cue, or an accidental participation in a protest—because they’re so naïve that even their own hearts get lost in the city’s maze.

---

### 1. Main Characters

| Name | Age | Origin | Personality | Love‑Life Quirk |
|------|-----|--------|-------------|-----------------|
| **Erik** | 28 | Ahrweiler (wine country) | The eternal dreamer; thinks “schmuck” is a nut. | Always shows up with a *paper* love letter, forgetting he’s in a digital age. |
| **Max** | 27 | Lüneburg (scholarly vibes) | Pragmatic, but clueless about body language. | Takes “It’s just a ‘like’ on Instagram” as a commitment. |
| **Clara** | 26 | Berlin‑born | Quick‑wit, speaks German/English, lives in a tiny co‑op. | Loves techno, hates her phone, and thinks “Berlin love” means "free Wi‑Fi". |
| **Nina** | 27 | International (Brazil) | Passionate, loves street art, thinks romance is a hashtag. | Sees every awkward moment as “photo‑opportunity”. |

---

### 2. Setting & Tone

- **Berlin**: Café culture, Mauerpark, techno clubs, the Spree river, and the occasional “Oberkrug” (old beer garden).  
- **Tone**: Quick‑fire jokes, visual gags (Erik’s “hand‑made” gift that looks like a plant pot), and that classic German awkward silence stretched to a minute.  
- **Cultural nods**: Reference to the “Berliner Kindl”, “Wiener Schnitzel”, and Berlin’s notorious “Biergarten” etiquette.

---

### 3. Pilot Idea – “First Date on the River”

**Plot**  
Erik and Max decide to impress Clara by renting a boat on the Spree. Max, ever the organizer, brings a *DIY* “love kit”: a paper heart, a bouquet of plastic flowers, and a playlist of 90’s Eurodance.  
- **Missteps**:  
  1. Max forgets the boat’s “no music after 2 pm” rule → a crowd of startled fish.  
  2. Erik mis‑reads “Zuhause” (home) as “zuhause” (zoo) → he drops a stuffed bear in the water.  
  3. Clara’s phone keeps auto‑typing “Biergarten” as a romantic keyword → she thinks they’re all about beer.  

**End**  
The boat gets towed, but they make a “memory” photo on a floating sign that reads “Berlin—where love floats”. Clara laughs and agrees to a second date *provided* they bring actual drinks next time.

---

### 4. Episode Skeletons (Season 1)

| # | Title | Hook |
|---|-------|------|
| 1 | “Love in the Tunnels” | Erik tries to “understand” the underground, literally going down a subway tunnel with a love letter. |
| 2 | “Berlin Love Letters” | Max writes an email in German; auto‑correct turns it into a recipe. |
| 3 | “The Dating App Dilemma” | They swipe right on a “German for Beginners” group chat. |
| 4 | “Mauerpark Misadventure” | They attempt a spontaneous karaoke session; ends in a live protest. |
| 5 | “The Artist’s Crush” | Clara’s boyfriend is a graffiti artist; they try to be supportive but create a new public mural. |
| 6 | “Schlitz and Slippery” | Max and Erik go on a “slippery ice‑cream” date and keep falling into each other’s arms. |
| 7 | “The “Schmuck” Confusion” | They think a German “schmuck” is a decorative spoon and show it to Nina. |
| 8 | “Berlin’s Big Date” | The trio goes to a fancy dinner; Erik mistakenly brings a jar of fermented cabbage. |
| 9 | “Love in 3D” | They rent VR goggles; a virtual “Berlin” turns into a literal love maze. |
|10 | “Season Finale: The Wedding Plan” | They mistakenly book a “wedding hall” for a bachelor party; the bride is actually a stray cat. |

---

### 5. Quick‑Fire Comedy Hooks

- **Language Blunders**: “Ich liebe dich” → “I love ditch”.  
- **Cultural Shock**: Max buys a “Biergarten” and ends up in a beer garden of actual beer.  
- **Techno Romance**: Erik tries to “dance” to techno by mimicking a DJ’s hand‑signals—wants a beat, gets a “beat” as a drumstick.  
- **Dating Apps**: They use a “German‑only” app; every match is a "Vorspiel" (foreplay) that ends up being a handshake.  

---

**Takeaway**  
Keep the beats tight—quick one‑liner punchlines, the kind that would land on a Berlin wall, and let the city itself be the third character. Let their naivete be their charm; let their missteps be the laugh track that carries the episode. And always finish with a line that reminds the audience: In Berlin, even a love story can start with a wrong turn and end in a great *U-Bahn* ride.

```json
{
  "name": "Erik Müller",
  "age": 28,
  "personality": "Unwittingly optimistic dreamer, loves a good glass of wine more than a Wi‑Fi signal. He speaks with a gentle, poetic tone, often turning everyday observations into metaphors. Technologically clumsy yet endearingly earnest, Erik sees the world in “possible worlds” rather than “realities.”",
  "motivation": "To prove that a small‑town boy can thrive in Berlin and, most importantly, to find a partner who shares his quirky view that love is a canvas waiting to be painted.",
  "pastChildhood": "Grew up in Ahrweiler’s vineyards, where he learned to differentiate Riesling from Spätburgunder. He spent hours listening to his grandmother’s stories of old‑world romance and the “Schadenfreude” in a perfect German sunset.",
  "pastTeenage": "Attended a regional high school, excelled in German literature, but struggled with the new digital world. Joined a local youth choir, which sparked his love of music but left him terrified of large crowds. Took an online German‑for‑Business course to impress a crush, learning the phrase “Ich habe Hunger” only to later realize it meant “I’m hungry,” not “I’m in love.”",
  "pastYoungUpUntilNow": "Moved to Berlin after finishing a 4‑year apprenticeship in hospitality. Took a barista job in a quirky café on Prenzlauer Berg. Tried to learn slang in his spare time, but only succeeded in mastering “Kaffee mit Milch” and “Biergarten.” His first Berlin apartment was a tiny room that smelled like currywurst and laundry detergent.",
  "relationTo": {
    "self": "Believes himself to be a romantic at heart, but often misreads social cues.",
    "family": "Close to his parents, who run the family winery. They occasionally drop by Berlin, bringing homemade preserves that smell like home.",
    "relatives": "Aunt Sabine who owns a bookshop in Cologne; Uncle Jürgen who is a freelance photographer in Berlin’s Kreuzberg.",
    "friends": "Max (the pragmatic best friend) and a handful of fellow baristas who share his love of obscure 80s synth pop.",
    "coworkers": "Barista team who appreciate his jokes but are often annoyed by his frequent “wine references.”",
    "strangers": "Eager to make acquaintances, yet frequently mistakes a stranger’s friendly gesture for a romantic interest.",
    "authorityFigures": "Never quite comfortable with strict rules—his first major boss once asked him to “keep it professional,” to which Erik replied, “Professionalism? I have a whole vineyard of experience.”",
    "locations": "Prenzlauer Berg (home & work), Mauerpark (frequent dating spots), Neukölln (his favorite currywurst stall), and the Spree River (his chosen, if misguided, venue for romance)."
  },
  "currentStateOf": {
    "physical": "Average build, always sporting a slightly wind‑touched mustache. Often has a coffee stain on his shirt from late‑night shifts.",
    "mental": "High curiosity, low skepticism—he believes almost everything he hears. Minor anxiety spikes when he has to interpret German slang or navigate Berlin’s public transport.",
    "spiritual": "A soft romantic idealist; often hums “Liebeslied” while making coffee, hoping to attract cosmic romance."
  },
  "strength": "Unstoppable optimism and an uncanny ability to see beauty where others see a problem. His sincerity and empathy win over people who otherwise would find his naivete off‑putting.",
  "weakness": "Overconfidence in his “world‑changing” ideas; poor grasp of German idioms leads to embarrassing misunderstandings; easily distracted by romance clichés.",
  "dailyRoutine": "Morning: wakes at 6:30, drinks a glass of Riesling for “morning motivation.” Mid‑morning: works as a barista, takes occasional “wine‑speak” breaks. Afternoon: strolls through Mauerpark, listening to street musicians, while mentally rehearsing what to say on his next date. Evening: tries a new recipe, watches a German film with subtitles, ends the night with a handwritten love note to the “city” before sleeping at 1 a.m.",
  "phobias": "Being alone in a dark apartment, public speaking (except when he’s in the kitchen explaining the difference between a dry Riesling and a sweet one), and accidentally starting a fire in the kitchen while making coffee.",
  "illness": {
    "physical": "None—though his heart rate spikes at the sight of a love‑song.",
    "mental": "Mild situational anxiety; occasionally experiences a “schadenfreude panic” when he over‑enthusiastically engages in a joke that backfires."
  }
}
```

```json
{
  "name": "Erkan Demir",
  "age": 28,
  "personality": "A gentle soul who carries Istanbul’s layered rhythm in his heartbeat. He blends the melodramatic flair of Turkish storytelling with a naïve optimism that often turns awkward into endearing. Technologically clumsy, he prefers handwritten notes to emojis, and he interprets every Berlin moment through the lens of a café‑filled Bosphorus sunset.",
  "motivation": "To prove that a Turkish dreamer can thrive in Berlin’s eclectic mosaic and, more importantly, to find a partner who shares his belief that love is a shared espresso and a shared conversation.",
  "pastChildhood": "Raised in a bustling Istanbul neighborhood where his parents owned a small kahvehan (coffee house). He spent childhood afternoons listening to his grandfather’s tales of the Golden Horn and learning how to balance a cup of Turkish coffee without spilling it.",
  "pastTeenage": "Attended a Turkish high school in Bakırköy. He excelled in literature and drama, often performing Shakespeare in Ottoman Turkish. At 16, he began learning German online to keep up with his future plans—his first German phrase was ‘Merhaba, wie geht es dir?’ which he pronounced with a heavy Istanbul accent, much to the amusement of his classmates.",
  "pastYoungUpUntilNow": "After graduating, he moved to Berlin to study German language and intercultural communication. He took a part‑time job in a kebab shop on Kreuzberg’s Boxhagener Platz, learning the nuances of German customer service while still serving döner with a smile. He lived in a shared flat with other students, where the smell of Turkish spice met the German bakery’s rye bread.",
  "relationTo": {
    "self": "Sees himself as a bridge between two cultures—he feels at home in both Istanbul’s chaotic streets and Berlin’s orderly lanes, but he’s still learning how to translate his emotions into German.",
    "family": "Parents run a small tea‑and‑bakery shop; they send him monthly shipments of Turkish sweets (lokum, baklava) and a handwritten postcard that says ‘Gel, aşk bizimle!’ (Come, love is with us).",
    "relatives": "Aunt Nazlı, who owns a bakery in Üsküdar; Uncle Can, a taxi driver who loves to talk about Berlin’s history. They all visit occasionally via the Bosphorus ferry, bringing a touch of home.",
    "friends": "Max (the practical best friend) and a handful of fellow baristas who love Turkish music and often play “Tükenden Şiir” (a modern Turkish hip‑hop rap).",
    "coworkers": "Kebab shop crew who appreciate his enthusiasm but sometimes tease him when he attempts to order in German with a Turkish accent.",
    "strangers": "Eager to meet new people, he frequently misinterprets friendly gestures as romantic interest, leading to comical situations.",
    "authorityFigures": "Never quite comfortable with strict German rules—he once responded to a boss’s “Professionalität” with a poetic line about the aroma of Turkish coffee.",
    "locations": "Berlin’s Kreuzberg (home & work), Istanbul’s Karaköy (flashbacks in flash‑forward scenes), Mauerpark (frequent dates), and the Spree River (his chosen yet misguided venue for romance)."
  },
  "currentStateOf": {
    "physical": "Tall and wiry, with a slight stooped posture from carrying heavy grocery bags during his earlier years. His hair is usually in a messy bun, a habit from his days mixing Turkish tea.",
    "mental": "High curiosity, low skepticism—he sees almost everything as a story waiting to happen. Minor anxiety spikes when interpreting German slang or navigating Berlin’s public transport.",
    "spiritual": "A soft romantic idealist; he often hums Turkish lullabies while making coffee, hoping to attract cosmic romance."
  },
  "strength": "Unstoppable optimism, a natural storyteller, and an innate ability to bring warmth into cold spaces—qualities that win over people who otherwise would find his naivete off‑putting.",
  "weakness": "Overconfidence in his “world‑changing” ideas; poor grasp of German idioms leads to embarrassing misunderstandings; easily distracted by romance clichés, especially those he hears in Turkish dramas.",
  "dailyRoutine": "Morning: wakes at 5:30, drinks a glass of Turkish tea for “morning motivation.” Mid‑morning: works as a barista, taking occasional “coffee‑speak” breaks. Afternoon: strolls through Mauerpark, listening to street musicians, while mentally rehearsing what to say on his next date. Evening: tries a new recipe, watches a German film with Turkish subtitles, ends the night with a handwritten love note to the “city” before sleeping at 1 a.m.",
  "phobias": "Being alone in a dark apartment, public speaking (except when he’s in the kitchen explaining the difference between a dry Riesling and a sweet one), and accidentally starting a fire in the kitchen while making coffee.",
  "illness": {
    "physical": "None—though his heart rate spikes at the sight of a love‑song.",
    "mental": "Mild situational anxiety; occasionally experiences a “schadenfreude panic” when he over‑enthusiastically engages in a joke that backfires."
  }
}
```

```json
{
  "name": "Erkan Demir",
  "age": 28,
  "personality": "Calm, introspective, and deeply empathetic. Erkan is a quiet observer who reads people’s body language like an open book. He values authenticity over pretense, often opting for genuine conversation instead of surface‑level small talk. While he’s not flashy, he brings a quiet confidence that draws people in.",
  "motivation": "To bridge cultural divides by living authentically in a new city, while simultaneously discovering whether love can be built on shared values rather than surface attractions.",
  "pastChildhood": "Raised in the historic quarter of Kadiköy, Istanbul, in a single‑parent household where his mother ran a modest grocery store. He learned resilience early, juggling schoolwork with helping out at the store and volunteering at a community youth center.",
  "pastTeenage": "At 15, he founded a student‑run podcast that discussed contemporary issues in Turkish society, often interviewing activists and artists. The podcast, titled *Günlük* (Daily), helped him develop a nuanced understanding of dialogue and narrative structure.",
  "pastYoungUpUntilNow": "After completing a bachelor’s degree in Comparative Literature, he spent a year in Berlin as a research assistant on a cross‑cultural study. He worked part‑time in a community kitchen where he cooked dishes from both cultures, learning the value of exchange and compromise.",
  "relationTo": {
    "self": "View himself as a translator of emotions, not just languages.",
    "family": "His mother is a school teacher; they maintain regular video calls that keep him grounded. They value education and humility over wealth.",
    "relatives": "Uncle Ahmet is a musician who introduced him to indie rock, while his cousin Leyla is a freelance graphic designer who often critiques his artwork.",
    "friends": "Max, his longtime friend from Turkey who now works as a graphic designer in Berlin; Maya, a fellow researcher who shares his interest in inter‑cultural studies.",
    "coworkers": "In his current role at a cultural nonprofit, he collaborates closely with a team that includes a sociologist, a photographer, and an immigrant rights activist.",
    "strangers": "He is cautious with strangers but opens up quickly to those who share his passion for thoughtful conversation.",
    "authorityFigures": "He respects authority but is quick to question policies that conflict with his moral compass, reflecting a strong ethical framework.",
    "locations": "Berlin’s Kreuzberg (residence & work), Istanbul’s Kadiköy (flashbacks), and occasional trips to the countryside to reconnect with his heritage."
  },
  "currentStateOf": {
    "physical": "Average height, a lean build, often dressed in simple, functional clothing that reflects his practical lifestyle.",
    "mental": "High curiosity, moderate anxiety in unfamiliar social settings; he practices mindfulness to maintain emotional balance.",
    "spiritual": "A spiritual wanderer, he incorporates elements of Sufi meditation and quiet reflection into his daily routine."
  },
  "strength": "Exceptional listening skills and a genuine desire to understand others, coupled with an ability to see patterns in cultural narratives.",
  "weakness": "Tendency to overthink, which sometimes delays decisive action; discomfort with overt displays of emotion that can be misinterpreted by those around him.",
  "dailyRoutine": "Morning: starts with a 10‑minute meditation, followed by a cup of strong Turkish coffee. Mid‑morning: research or volunteer work. Afternoon: engages in creative projects (writing, podcasting). Evening: attends a local meetup or cultural event; ends with journaling in German and Turkish to practice language fluency.",
  "phobias": "Public speaking in unfamiliar settings, being misunderstood due to cultural differences, and losing touch with his roots.",
  "illness": {
    "physical": "No chronic conditions, occasional mild seasonal allergies.",
    "mental": "Anxiety that surfaces during large, unfamiliar gatherings; he manages it through structured routines and supportive friendships."
  }
}
```